In [ ]:
import os
from timeit import default_timer as timer
import pandas as pd
from datetime import datetime, timedelta
from glob import glob
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
path_to_data='/scratch/spf248/covid/data'
start_date='01/01/2020'
end_date='04/16/2020'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
city_id='UC_NM_MN'
city_name='Jakarta'

# Import Data

In [ ]:
print('Import')
start = timer()
data_dirs=glob(os.path.join(path_to_data,source,'processed',country,'pings_geocoded','*'))
pings_geocoded=pd.concat(
    pd.read_parquet(parquet_file)
    for data_dir in data_dirs for parquet_file in Path(data_dir).glob('*.parquet')
)
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Import')
start = timer()
admin2city=pd.read_excel(os.path.join(path_to_data,'admin',country,'city2adminGHSL.xlsx'))[[city_id,admin_id]].drop_duplicates(admin_id)
print("Done in", round(timer()-start), "sec")  

# Data processing

In [ ]:
print('Process')
start = timer()
pings_geocoded['date']=pd.to_datetime(pings_geocoded.time.dt.date)
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Select pings within city')
start = timer()
pings_city=pings_geocoded.merge(admin2city[admin2city[city_id]==city_name])[['time','date','cuebiq_id',admin_id]]
print("Done in", round(timer()-start), "sec")

In [ ]:
print('Process')
start = timer()
pings_city=pings_city[(pings_city.date>=pd.to_datetime(start_date))&(pings_city.date<=pd.to_datetime(end_date))].copy()
pings_city.sort_values(by=['cuebiq_id','time'],inplace=True)
pings_city.drop('time',1,inplace=True)
print("Done in", round(timer()-start), "sec")

# Count trips between admins

In [ ]:
print('Count Trips')
start = timer()
trips=pd.concat([pings_city.drop('cuebiq_id',1).rename(columns=lambda x:x+'_origin').reset_index(drop=True),
pings_city.groupby('cuebiq_id').shift(-1).rename(columns=lambda x:x+'_destination').reset_index(drop=True)],1)
trips=trips[trips.date_origin==trips.date_destination].copy()
trips=trips.groupby(['date_origin',admin_id+'_origin',admin_id+'_destination']
)['date_destination'].count().rename('trip').reset_index().rename(columns={'date_origin':'date'})
print("Done in", round(timer()-start), "sec")

In [ ]:
print('Save')
start = timer()
trips.to_csv(os.path.join(path_to_data,source,'processed',country,'OD_'+city_name+'.csv'))
print("Done in", round(timer()-start), "sec")

In [ ]:
trips.head()